In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
import math
from tqdm.auto import tqdm
import sys
import os
import scipy
from scipy.stats import linregress
import statsmodels.api as sm
from joblib import Parallel, delayed
import time
import h5py
from sympy import symbols, Function, Eq, solve, exp, diff, dsolve, simplify
from scipy.optimize import minimize

from Others import *

# Config

In [ ]:
ini = -2
config = {'N':200000,
    'real_A': 1, 'real_B': 1, 'real_C': 1, 'real_D': 1,
    'Q': 1, 'H': 1, 'x_0': 1, 'T': 1,
    'initial_A': ini, 'initial_B': ini, 
    'initial_C': ini, 'initial_D': ini, 
    'initial_nt': 100, 'initial_m_n':1, 
    'initial_v_n':5, 'initial_seed':None
}

In [ ]:
num_exp = 120
initial_seeds = list(range(1, num_exp + 1))

# Load Data

In [ ]:
file_path = '../Data_Model_Based/'

data = {}

with h5py.File(file_path + 'Algo_full_results.h5', 'r') as f:
    for seed, group in f.items():
        data[int(seed)] = {k: np.array(v) for k, v in group.items()}

# Check All Results
## Convergence Rate

In [ ]:
def plot_log_mse(data, star, title, log_iter_start, log_iter_end, lower_percentile, upper_percentile, initial_n):
    mse_all = (np.array(data) - star) ** 2
    avg_mse = np.mean(mse_all, axis=0)
    lower_bound = np.percentile(mse_all, lower_percentile, axis=0)
    upper_bound = np.percentile(mse_all, upper_percentile, axis=0)
    log_mse = np.log(avg_mse)
    log_iterations = np.log(np.arange(initial_n + 1, initial_n + len(avg_mse) + 1))
    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(log_iterations, log_mse, label='Log MSE')
    ax.fill_between(log_iterations, np.log(lower_bound), np.log(upper_bound), color='gray', alpha=0.3, label=f'{lower_percentile}th-{upper_percentile}th Percentile')

    # Perform regression in the specified range
    valid_indices = (log_iterations >= log_iter_start) & (log_iterations <= log_iter_end)
    slope, intercept, _, _, _ = linregress(log_iterations[valid_indices], log_mse[valid_indices])

    # Compute regression line for **all** log_iterations
    full_regression_line = slope * log_iterations + intercept

    # Plot regression line across all log iterations
    ax.plot(log_iterations, full_regression_line, 'r', label=f'Linear Regression\ny = {slope:.2f}x {"+" if intercept >= 0 else "-"} {abs(intercept):.2f}')
    
    # Formatting updates
    ax.set_xlabel('Log Episodes', fontsize=18)
    ax.set_ylabel(f'Log MSE of {title}', fontsize=18)
    ax.tick_params(axis='both', labelsize=18)  # Increase tick label size
    ax.legend(fontsize=16)  # Increase legend font size
    ax.set_title("")  # Set title to empty
    ax.grid(True)
    
    return fig

In [ ]:
phi_1_values = np.array([result['phi_1_list'] for result in data.values()])
fig1 = plot_log_mse(phi_1_values, -2, '$\phi_1$', np.log(5000), np.log(200000), 25, 75, 0)

In [ ]:
fig1.savefig('phi1_log_mse_model-based.png')

## Regret

In [ ]:
def plot_log_regret(regrets, title, log_iter_start, log_iter_end, lower_percentile, upper_percentile, initial_n):
    all_regrets = [np.cumsum(regret) for regret in regrets]

    avg_regret = np.median(all_regrets, axis=0)
    lower_bound = np.percentile(all_regrets, lower_percentile, axis=0)
    upper_bound = np.percentile(all_regrets, upper_percentile, axis=0)

    log_avg_regret = np.log(avg_regret)
    log_iterations = np.log(np.arange(initial_n + 1, initial_n + len(log_avg_regret) + 1))

    fig, ax = plt.subplots(figsize=(8, 6))  # Consistent figure size
    ax.plot(log_iterations, log_avg_regret, label='Log Expected Regret')
    ax.fill_between(log_iterations, np.log(lower_bound), np.log(upper_bound), color='gray', alpha=0.3, label=f'{lower_percentile}th-{upper_percentile}th Percentile')

    # Perform regression in the specified range
    valid_indices = (log_iterations >= log_iter_start) & (log_iterations <= log_iter_end)
    slope, intercept, _, _, _ = linregress(log_iterations[valid_indices], log_avg_regret[valid_indices])

    # Compute regression line for **all** log_iterations
    full_regression_line = slope * log_iterations + intercept

    # Plot regression line across all log iterations
    ax.plot(log_iterations, full_regression_line, 'r', label=f'Linear Regression\ny = {slope:.2f}x {"+" if intercept >= 0 else "-"} {abs(intercept):.2f}')

    # Formatting updates
    ax.set_xlabel('Log Episodes', fontsize=18)
    ax.set_ylabel('Log Expected Regret', fontsize=18)
    ax.tick_params(axis='both', labelsize=18)
    ax.legend(fontsize=16)
    ax.set_title("")  # Set title to empty for consistency
    ax.grid(True)

    return fig, all_regrets

In [ ]:
regrets = np.array([result['regrets'] for result in data.values()])
fig2, all_regrets = plot_log_regret(regrets, '', np.log(5000), np.log(200000), 25, 75, 0)

In [ ]:
fig2.savefig('log_regrets_model-based.png')